In [8]:
# !pip install tabulate
# !pip install psutil
# !pip install elasticsearch

In [10]:
import platform
import psutil
import os
from datetime import datetime
from tabulate import tabulate

import elasticsearch
from elasticsearch import Elasticsearch

import json

In [11]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [39]:
es.ping()

True

In [49]:
es.indices.create(index='pid_index', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'pid_index'}

In [64]:
#es.indices.delete(index='pid_index', ignore=400)

In [51]:
es.info()

{'name': 'arpit',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'lRupjgnrQGCK1Ataaqeuyg',
 'version': {'number': '7.16.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb',
  'build_date': '2021-12-18T19:42:46.604893745Z',
  'build_snapshot': False,
  'lucene_version': '8.10.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [52]:
print('General info about processes')
psutil.test()

General info about processes
USER         PID  %MEM     VSZ     RSS  NICE STATUS  START   TIME  CMDLINE
root           1   0.1  166.0M    6.2M        sleep  00:29  34:32  /sbin/init sp
root           2   0.0    0.0B    0.0B        sleep  00:29  30:02  kthreadd
root           3   0.0    0.0B    0.0B   -20   idle  00:29  30:00  rcu_gp
root           4   0.0    0.0B    0.0B   -20   idle  00:29  30:00  rcu_par_gp
root           6   0.0    0.0B    0.0B   -20   idle  00:29  30:00  kworker/0:0H-
root           9   0.0    0.0B    0.0B   -20   idle  00:29  30:00  mm_percpu_wq
root          10   0.0    0.0B    0.0B        sleep  00:29  30:00  rcu_tasks_rud
root          11   0.0    0.0B    0.0B        sleep  00:29  30:00  rcu_tasks_tra
root          12   0.0    0.0B    0.0B        sleep  00:29  30:19  ksoftirqd/0
root          13   0.0    0.0B    0.0B         idle  00:29  30:24  rcu_sched
root          14   0.0    0.0B    0.0B        sleep  00:29  30:02  migration/0
root          15   0.0    0.0

sherlock   2405864   0.3   25.2G   31.9M        sleep  14:57  30:00  /snap/chrom
root       2463819   0.0    0.0B    0.0B         idle  14:59  30:00  kworker/2:2
root       2474425   0.0    0.0B    0.0B         idle  14:59  30:00  kworker/0:1
sherlock   2520673   0.2  424.7M   26.5M        sleep  15:00  30:03  /usr/libexe
root       2538688   0.0    0.0B    0.0B         idle  15:01  30:00  kworker/u8:
sherlock   2732210   2.3   25.3G  271.5M        sleep  11:33  30:36  /snap/chrom
root       2915062   0.0    0.0B    0.0B   -20   idle  11:38  30:13  kworker/u9:
root       3652197   0.0    0.0B    0.0B         idle  13:44  30:33  kworker/1:1
root       3718817   0.0    0.0B    0.0B   -20   idle  12:00  30:13  kworker/u9:
root       3952051   0.0    0.0B    0.0B         idle  13:51  30:10  kworker/3:1


In [66]:
sys_mem = psutil.virtual_memory().total #+ psutil.swap_memory().total
# used_mem = psutil.virtual_memory().used
print("Total Memory of this system is", round(sys_mem/(1024**3), 3), 'GB')
# print("Total Used Memory of this system right now is", round(used_mem/(1024**3), 3), 'GB')
# print("Percentage of memory used of this system right now is", round(used_mem/sys_mem*100, 3), '%')
print("Total CPU Usage of this system right now is", psutil.cpu_percent(), '%')

Total Memory of this system is 11.615 GB
Total CPU Usage of this system right now is 42.8 %


In [54]:
li = []
l_small = []

for process in psutil.process_iter():
    pid = process.pid                     # getting pid
    cpu = process.cpu_percent()           # getting cpu percentage
    
    try:
        mem = process.memory_full_info()  # getting whole memory info
        mem_uss = mem.uss                 # resident set size(total memory except swap)
        mem_rss = mem.rss                 # unique set size(private memory)
    except psutil.AccessDenied:           # this access is denied for root files
        mem = 0
        mem_uss = 0
        mem_rss = 0        
            
    total_mem_p = round(mem_rss / sys_mem * 100, 3)  # calculating total memory percange for each process
    total_mem_mb = round(mem_rss/(1024**2), 3)       # converting total memory for each process from byte to MB
    uss_mem_p = round(mem_uss / sys_mem * 100, 3)    # calculating USS memory percange for each process
    
    # appending info to list
    li.append([pid, cpu, total_mem_p, total_mem_mb, uss_mem_p, process])
    l_small.append([pid, cpu, total_mem_p, total_mem_mb, uss_mem_p])
    
    # creating dictionary to add to elastic search 
    data = {'pid': pid,'CPU%': cpu, 'TotalMemory%': total_mem_p, 'TotalMemory_in_MB': total_mem_mb, 'USS_Memory%': uss_mem_p}#, 'Process_Detail': process}
    
    # adding process information to ElasticSearch Database
    res_u = es.index(index='pid_index', document=data, id=pid)
    print("Saved to ElasticDatabase for pid", pid)
    print(res_u)
    
print('All Saved to Elastic database')

Saved to ElasticDatabase for pid 1
{'_index': 'pid_index', '_type': '_doc', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Saved to ElasticDatabase for pid 2
{'_index': 'pid_index', '_type': '_doc', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
Saved to ElasticDatabase for pid 3
{'_index': 'pid_index', '_type': '_doc', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
Saved to ElasticDatabase for pid 4
{'_index': 'pid_index', '_type': '_doc', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
Saved to ElasticDatabase for pid 6
{'_index': 'pid_index', '_type': '_doc', '_id': '6', '_version': 1, 'result': 'created', '_shards': {'total': 2, 

Saved to ElasticDatabase for pid 95
{'_index': 'pid_index', '_type': '_doc', '_id': '95', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 40, '_primary_term': 1}
Saved to ElasticDatabase for pid 96
{'_index': 'pid_index', '_type': '_doc', '_id': '96', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 41, '_primary_term': 1}
Saved to ElasticDatabase for pid 97
{'_index': 'pid_index', '_type': '_doc', '_id': '97', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 42, '_primary_term': 1}
Saved to ElasticDatabase for pid 98
{'_index': 'pid_index', '_type': '_doc', '_id': '98', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 43, '_primary_term': 1}
Saved to ElasticDatabase for pid 99
{'_index': 'pid_index', '_type': '_doc', '_id': '99', '_version': 1, 'result': 'created', '_shards':

Saved to ElasticDatabase for pid 327
{'_index': 'pid_index', '_type': '_doc', '_id': '327', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 80, '_primary_term': 1}
Saved to ElasticDatabase for pid 329
{'_index': 'pid_index', '_type': '_doc', '_id': '329', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 81, '_primary_term': 1}
Saved to ElasticDatabase for pid 332
{'_index': 'pid_index', '_type': '_doc', '_id': '332', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 82, '_primary_term': 1}
Saved to ElasticDatabase for pid 334
{'_index': 'pid_index', '_type': '_doc', '_id': '334', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 83, '_primary_term': 1}
Saved to ElasticDatabase for pid 338
{'_index': 'pid_index', '_type': '_doc', '_id': '338', '_version': 1, 'result': 'created', 

Saved to ElasticDatabase for pid 988
{'_index': 'pid_index', '_type': '_doc', '_id': '988', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 118, '_primary_term': 1}
Saved to ElasticDatabase for pid 989
{'_index': 'pid_index', '_type': '_doc', '_id': '989', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 119, '_primary_term': 1}
Saved to ElasticDatabase for pid 992
{'_index': 'pid_index', '_type': '_doc', '_id': '992', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 120, '_primary_term': 1}
Saved to ElasticDatabase for pid 996
{'_index': 'pid_index', '_type': '_doc', '_id': '996', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 121, '_primary_term': 1}
Saved to ElasticDatabase for pid 997
{'_index': 'pid_index', '_type': '_doc', '_id': '997', '_version': 1, 'result': 'create

Saved to ElasticDatabase for pid 6260
{'_index': 'pid_index', '_type': '_doc', '_id': '6260', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 157, '_primary_term': 1}
Saved to ElasticDatabase for pid 6284
{'_index': 'pid_index', '_type': '_doc', '_id': '6284', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 158, '_primary_term': 1}
Saved to ElasticDatabase for pid 6293
{'_index': 'pid_index', '_type': '_doc', '_id': '6293', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 159, '_primary_term': 1}
Saved to ElasticDatabase for pid 6302
{'_index': 'pid_index', '_type': '_doc', '_id': '6302', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 160, '_primary_term': 1}
Saved to ElasticDatabase for pid 6313
{'_index': 'pid_index', '_type': '_doc', '_id': '6313', '_version': 1, 'result

Saved to ElasticDatabase for pid 7200
{'_index': 'pid_index', '_type': '_doc', '_id': '7200', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 195, '_primary_term': 1}
Saved to ElasticDatabase for pid 7202
{'_index': 'pid_index', '_type': '_doc', '_id': '7202', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 196, '_primary_term': 1}
Saved to ElasticDatabase for pid 7204
{'_index': 'pid_index', '_type': '_doc', '_id': '7204', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 197, '_primary_term': 1}
Saved to ElasticDatabase for pid 7208
{'_index': 'pid_index', '_type': '_doc', '_id': '7208', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 198, '_primary_term': 1}
Saved to ElasticDatabase for pid 7210
{'_index': 'pid_index', '_type': '_doc', '_id': '7210', '_version': 1, 'result

Saved to ElasticDatabase for pid 174645
{'_index': 'pid_index', '_type': '_doc', '_id': '174645', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 234, '_primary_term': 1}
Saved to ElasticDatabase for pid 175085
{'_index': 'pid_index', '_type': '_doc', '_id': '175085', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 235, '_primary_term': 1}
Saved to ElasticDatabase for pid 177389
{'_index': 'pid_index', '_type': '_doc', '_id': '177389', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 236, '_primary_term': 1}
Saved to ElasticDatabase for pid 184671
{'_index': 'pid_index', '_type': '_doc', '_id': '184671', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 237, '_primary_term': 1}
Saved to ElasticDatabase for pid 184680
{'_index': 'pid_index', '_type': '_doc', '_id': '184680', '_

Saved to ElasticDatabase for pid 1981966
{'_index': 'pid_index', '_type': '_doc', '_id': '1981966', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 271, '_primary_term': 1}
Saved to ElasticDatabase for pid 2015061
{'_index': 'pid_index', '_type': '_doc', '_id': '2015061', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 272, '_primary_term': 1}
Saved to ElasticDatabase for pid 2074755
{'_index': 'pid_index', '_type': '_doc', '_id': '2074755', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 273, '_primary_term': 1}
Saved to ElasticDatabase for pid 2186789
{'_index': 'pid_index', '_type': '_doc', '_id': '2186789', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 274, '_primary_term': 1}
Saved to ElasticDatabase for pid 2220753
{'_index': 'pid_index', '_type': '_doc', '_id': '22

In [55]:
print('Printing CPU% and Memory%\n')
print(tabulate(l_small, headers=["pID", "CPU%", "TotalMemory%", "TotalMemory_in_MB", "USS_Memory%"]))

Printing CPU% and Memory%

    pID    CPU%    TotalMemory%    TotalMemory_in_MB    USS_Memory%
-------  ------  --------------  -------------------  -------------
      1     0             0                    0              0
      2     0             0                    0              0
      3     0             0                    0              0
      4     0             0                    0              0
      6     0             0                    0              0
      9     0             0                    0              0
     10     0             0                    0              0
     11     0             0                    0              0
     12     0.1           0                    0              0
     13     0.2           0                    0              0
     14     0             0                    0              0
     15     0             0                    0              0
     16     0             0                    0              0
     

In [56]:
print('Printing CPU% and Memory% with Process details')
print(tabulate(li, headers=["pID", "CPU%", "TotalMemory%", "TotalMemory_in_MB", "USS_Memory%", "Process_Detail"]))

Printing CPU% and Memory% with Process details
    pID    CPU%    TotalMemory%    TotalMemory_in_MB    USS_Memory%  Process_Detail
-------  ------  --------------  -------------------  -------------  -----------------------------------------------------------------------------------------------------------
      1     0             0                    0              0      psutil.Process(pid=1, name='systemd', status='sleeping', started='00:29:08')
      2     0             0                    0              0      psutil.Process(pid=2, name='kthreadd', status='sleeping', started='00:29:08')
      3     0             0                    0              0      psutil.Process(pid=3, name='rcu_gp', status='idle', started='00:29:08')
      4     0             0                    0              0      psutil.Process(pid=4, name='rcu_par_gp', status='idle', started='00:29:08')
      6     0             0                    0              0      psutil.Process(pid=6, name='kworker/0:0H-eve

In [58]:
# checking elasticDatabase
result = es.search(index='pid_index')

In [59]:
result

{'took': 863,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 295, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'pid_index',
    '_type': '_doc',
    '_id': '101',
    '_score': 1.0,
    '_source': {'pid': 101,
     'CPU%': 0.0,
     'TotalMemory%': 0.0,
     'TotalMemory_in_MB': 0.0,
     'USS_Memory%': 0.0}},
   {'_index': 'pid_index',
    '_type': '_doc',
    '_id': '103',
    '_score': 1.0,
    '_source': {'pid': 103,
     'CPU%': 0.0,
     'TotalMemory%': 0.0,
     'TotalMemory_in_MB': 0.0,
     'USS_Memory%': 0.0}},
   {'_index': 'pid_index',
    '_type': '_doc',
    '_id': '105',
    '_score': 1.0,
    '_source': {'pid': 105,
     'CPU%': 1.2,
     'TotalMemory%': 0.0,
     'TotalMemory_in_MB': 0.0,
     'USS_Memory%': 0.0}},
   {'_index': 'pid_index',
    '_type': '_doc',
    '_id': '106',
    '_score': 1.0,
    '_source': {'pid': 106,
     'CPU%': 0.0,
     'TotalMemory%': 0.0,
     'TotalM

In [62]:
# checking elasticDatabase for particular pid which was not visibile in prvious cell
result = es.search(index='pid_index', query={'match': {'pid': 6379}})

In [63]:
result

{'took': 250,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'pid_index',
    '_type': '_doc',
    '_id': '6379',
    '_score': 1.0,
    '_source': {'pid': 6379,
     'CPU%': 12.7,
     'TotalMemory%': 0.501,
     'TotalMemory_in_MB': 59.57,
     'USS_Memory%': 0.454}}]}}

In [96]:
elastic_li = []

for item in l_small:
    result = es.search(index='pid_index', query={'match': {'pid': item[0]}})
    elastic_li.append(result)
    
elastic_li

[{'took': 0,
  'timed_out': False,
  '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 1, 'relation': 'eq'},
   'max_score': 1.0,
   'hits': [{'_index': 'pid_index',
     '_type': '_doc',
     '_id': '1',
     '_score': 1.0,
     '_source': {'pid': 1,
      'CPU%': 0.0,
      'TotalMemory%': 0.0,
      'TotalMemory_in_MB': 0.0,
      'USS_Memory%': 0.0}}]}},
 {'took': 0,
  'timed_out': False,
  '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 1, 'relation': 'eq'},
   'max_score': 1.0,
   'hits': [{'_index': 'pid_index',
     '_type': '_doc',
     '_id': '2',
     '_score': 1.0,
     '_source': {'pid': 2,
      'CPU%': 0.0,
      'TotalMemory%': 0.0,
      'TotalMemory_in_MB': 0.0,
      'USS_Memory%': 0.0}}]}},
 {'took': 0,
  'timed_out': False,
  '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 1, 'relation': 'eq'},
   'max_score': 1.0,
 

In [97]:
json.dumps(elastic_li)

'[{"took": 0, "timed_out": false, "_shards": {"total": 1, "successful": 1, "skipped": 0, "failed": 0}, "hits": {"total": {"value": 1, "relation": "eq"}, "max_score": 1.0, "hits": [{"_index": "pid_index", "_type": "_doc", "_id": "1", "_score": 1.0, "_source": {"pid": 1, "CPU%": 0.0, "TotalMemory%": 0.0, "TotalMemory_in_MB": 0.0, "USS_Memory%": 0.0}}]}}, {"took": 0, "timed_out": false, "_shards": {"total": 1, "successful": 1, "skipped": 0, "failed": 0}, "hits": {"total": {"value": 1, "relation": "eq"}, "max_score": 1.0, "hits": [{"_index": "pid_index", "_type": "_doc", "_id": "2", "_score": 1.0, "_source": {"pid": 2, "CPU%": 0.0, "TotalMemory%": 0.0, "TotalMemory_in_MB": 0.0, "USS_Memory%": 0.0}}]}}, {"took": 0, "timed_out": false, "_shards": {"total": 1, "successful": 1, "skipped": 0, "failed": 0}, "hits": {"total": {"value": 1, "relation": "eq"}, "max_score": 1.0, "hits": [{"_index": "pid_index", "_type": "_doc", "_id": "3", "_score": 1.0, "_source": {"pid": 3, "CPU%": 0.0, "TotalMemor